In [1]:
from ase.io import read, write, Trajectory
import numpy as np

In [2]:
def CoM(clus):
    """
    Support function to set the origin of the cluster at the centre of the mass
    """
    (cx, cy, cz) = clus.get_center_of_mass()
    new_xyz = []
    for i, a in enumerate(clus):
        x, y, z = a.position
        x -= cx
        y -= cy
        z -= cz
        new_xyz.append((x, y, z))
    clus.set_positions(new_xyz)
    return clus

In [3]:
def checkSimilar(clus1, clus2):

    """Check whether two clusters are similar or not by comparing their moments of inertia"""
    Inertia1 = clus1.get_moments_of_inertia()
    Inertia2 = clus2.get_moments_of_inertia()
    # print(Inertia1, Inertia2, 'diff: ', Inertia1-Inertia2)

    tol = 0.01
    if (
        Inertia1[0] * (1 - tol) <= Inertia2[0] <= Inertia1[0] * (1 + tol)
        and Inertia1[1] * (1 - tol) <= Inertia2[1] <= Inertia1[1] * (1 + tol)
        and Inertia1[2] * (1 - tol) <= Inertia2[2] <= Inertia1[2] * (1 + tol)
    ):
        similar = True
    else:
        similar = False

    return similar

In [4]:
ini_gen = 0
end_gen = 15

In [5]:
ene_tol = 0.02
dist_tol = 0.02

num_check_similar = 0
diff_dist_abs_sum_list = []

non_similar_clus = []
similar_clus = []
non_similar_clus_ene_list = []
similar_clus_ene_list = []
non_similar_clus_diff_dist = []
similar_clus_diff_dist = []

n_traj1_list = []
n_traj2_list = []

for k in range(ini_gen, end_gen):
    file1  = 'mut_after_relax_gen'+str(k+1)+'.traj'
    file2  = 'vasp_mut_after_relax_gen'+str(k+1)+'.traj'

    traj1 = Trajectory(file1)
    traj2 = Trajectory(file2)
    
    n_traj1 = len(traj1)
    n_traj2 = len(traj2)
    n_traj1_list.append(n_traj1)
    n_traj2_list.append(n_traj2)
    
    for i in range(len(traj1)):
        num_check_similar = num_check_similar + 1
        clus1 = traj1[i]
        clus2 = traj2[i]
        
        ene1 = clus1.get_potential_energy()
        ene2 = clus2.get_potential_energy()
        diff_ene = ene1 - ene2
        
        dist1 = clus1.get_all_distances() 
        dist2 = clus2.get_all_distances()
        diff_dist = dist1 - dist2
        diff_dist_abs = np.absolute(diff_dist)
        diff_dist_avg = np.sum(diff_dist_abs) / len(diff_dist)**2
        
        inertia1 = clus1.get_moments_of_inertia()
        inertia2 = clus2.get_moments_of_inertia()
        diff_inertia = inertia1 - inertia2
              
        if checkSimilar(clus1, clus2) == True:
            if diff_dist_avg > dist_tol:
                if abs(diff_ene) > ene_tol:
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                    
            else:
                similar_clus.append((k,i))
                similar_clus_ene_list.append(diff_ene)
                similar_clus_diff_dist.append(diff_dist_avg)
                #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                
        else:
            if diff_dist_avg < dist_tol:
                if abs(diff_ene) < ene_tol:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else: 
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
            else:
                non_similar_clus.append((k,i))
                non_similar_clus_ene_list.append(diff_ene)
                non_similar_clus_diff_dist.append(diff_dist_avg)
                print((k, i),diff_dist_avg, diff_ene, diff_inertia )

print(len(non_similar_clus))
print(len(similar_clus))
print(num_check_similar)

#print(sorted(non_similar_clus_ene_list))
#print(non_similar_clus)
#print(sorted(non_similar_clus_ene_list))
#print(sorted(non_similar_clus_diff_dist))

print(n_traj1_list)
print(n_traj2_list)

(0, 4) 0.3829350526789102 0.8438938699999987 [  55.34476686 -107.57079545   15.6247472 ]
(0, 6) 0.48078177238738457 0.04226351999999878 [ 8.91589021 -6.06943159  7.504038  ]
(1, 0) 0.41010408814389177 0.07475517999999681 [ 38.04597271 -36.74084619   2.50749286]
(2, 2) 0.5030090695482683 0.23009047000000038 [ 31.70116898 -34.75527735   5.88471715]
(2, 6) 0.7798445311208219 0.0780812200000014 [ 42.3965625  -35.26820876   7.46054993]
(2, 7) 0.06205161701999681 -0.030998849999999578 [ -2.75209453   7.56853722 -11.98885785]
(3, 1) 0.5338442024667881 -0.16797320999999954 [-49.93008574  33.29888909  -4.90982251]
(3, 5) 0.30325763745608053 -0.005805850000001556 [-5.20003258  1.65425867 -3.29087645]
(3, 7) 0.24542304219036812 -0.009343109999999655 [-6.21223821  5.660499   -0.38198196]
(3, 8) 0.316618189211204 1.0076844399999985 [-97.98010686 118.71195561 117.82490612]
(4, 5) 0.9871275289959178 4.1844864699999995 [211.36917356 216.57738254 449.54148428]
(5, 1) 0.7373371474970681 -1.3112742500000

In [6]:
filename="clus_Al7.log"

parent_calls_list = []
with open(filename, 'r') as fh:
    for line in fh:
        if line.startswith("Parent"):
            line = line.strip()
            words = line.split(':')
            words[1] = words[1].lstrip()
            parent_calls_list.append(words[1])
#print(parent_calls_list)
#print((parent_calls_list[0]))

item_split = []
for item in parent_calls_list:
    print( item)
    item_split.append(item.split(","))
#print(item_split)

#print(item_split[0])


[40, 17, 7, 23, 24, 6, 46, 30, 66, 22]
[19, 8]
[16, 13, 35, 20, 40, 8, 67, 10, 14, 25]
[14, 17, 27, 7, 8, 49, 36, 32, 8, 18]
[10, 11, 37, 25, 10, 31]
[6, 20, 8, 53]
[39, 7, 27, 32, 16, 32, 14, 26, 16, 21]
[25, 8, 22, 29, 22, 27]
[36, 37, 17, 14, 19, 8, 37, 33, 20, 30]
[7, 15, 8, 40]
[27, 6, 6, 19, 18]
[30, 7, 32, 6, 23, 25, 37, 37, 29, 20]
[7, 28, 12, 30, 14, 19, 21, 26, 9, 28]
[16, 10, 37, 13, 19, 10, 16, 21, 9, 41]
[8, 31, 33, 10]


In [7]:
new_vasp_calls = []
len_new_vasp_calls = []
for i in range(len(item_split)):
    k=len(item_split[i])
    new_int_list = []
    
    a = list(item_split[i][0].lstrip())
    del a[0] 
    a = int(''.join(a))
    new_int_list.append(a)
    #print(a)
    for k in range(1,(len(item_split[i])-1)):
        b = int(item_split[i][k])
        new_int_list.append(b)
        #print(b)  
    c = list(item_split[i][-1].lstrip())
    del c[-1]
    c = int(''.join(c))
    new_int_list.append(c)
    #print(c)
    #print(new_int_list)
    len_new_list = len(new_int_list)
    new_vasp_calls.append(new_int_list)
    len_new_vasp_calls.append(len_new_list)
print(new_vasp_calls)
print(len(new_vasp_calls))
print(len_new_vasp_calls)
print(sum(len_new_vasp_calls))

[[40, 17, 7, 23, 24, 6, 46, 30, 66, 22], [19, 8], [16, 13, 35, 20, 40, 8, 67, 10, 14, 25], [14, 17, 27, 7, 8, 49, 36, 32, 8, 18], [10, 11, 37, 25, 10, 31], [6, 20, 8, 53], [39, 7, 27, 32, 16, 32, 14, 26, 16, 21], [25, 8, 22, 29, 22, 27], [36, 37, 17, 14, 19, 8, 37, 33, 20, 30], [7, 15, 8, 40], [27, 6, 6, 19, 18], [30, 7, 32, 6, 23, 25, 37, 37, 29, 20], [7, 28, 12, 30, 14, 19, 21, 26, 9, 28], [16, 10, 37, 13, 19, 10, 16, 21, 9, 41], [8, 31, 33, 10]]
15
[10, 2, 10, 10, 6, 4, 10, 6, 10, 4, 5, 10, 10, 10, 4]
111


In [8]:
print(len(similar_clus))
print(similar_clus)

68
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9), (1, 1), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 8), (2, 9), (3, 0), (3, 2), (3, 3), (3, 4), (3, 6), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 0), (5, 2), (5, 3), (6, 1), (6, 4), (6, 7), (6, 9), (7, 1), (7, 2), (7, 3), (8, 2), (8, 5), (8, 6), (8, 7), (9, 0), (9, 1), (9, 2), (9, 3), (10, 0), (10, 1), (10, 2), (10, 3), (11, 1), (11, 2), (11, 3), (11, 4), (11, 7), (11, 9), (12, 0), (12, 2), (12, 5), (12, 6), (12, 8), (13, 0), (13, 1), (13, 2), (13, 3), (13, 5), (13, 7), (14, 0), (14, 3)]


In [9]:
file_vasp="results.log"

vasp_calls = []
vasp_calls_list = []
with open(file_vasp, 'r') as fh:
    total_images = []
    for line in fh:       
        if "VASP Calls" in line:
            line = line.strip()
            words = line.split(':')
            words[1] = int(words[1].lstrip())
            vasp_calls.append(words[1])
        if "Total images in the trajectory" in line:
            line = line.strip()
            img_num = line.split(':')
            img_num[1] = int(img_num[1].lstrip())
            total_images.append(img_num[1])

print(vasp_calls)
#print(len(vasp_calls))
#print(total_images)
#print(len(total_images))
#print(sum(total_images))
print('\n')

kk = 0
for i in total_images:
    traj_calls = []
    for j in range(kk, kk+i):
        traj_calls.append(vasp_calls[j])
    vasp_calls_list.append(traj_calls)
    kk = kk + i

print(vasp_calls_list)

[78, 20, 9, 37, 54, 8, 66, 61, 40, 39, 59, 9, 9, 76, 65, 29, 62, 21, 30, 13, 43, 145, 43, 29, 58, 10, 10, 61, 52, 31, 34, 53, 14, 12, 69, 65, 9, 53, 9, 14, 9, 50, 50, 35, 12, 50, 15, 48, 31, 16, 26, 46, 65, 9, 18, 12, 41, 14, 47, 30, 12, 27, 69, 20, 42, 36, 44, 43, 8, 44, 9, 76, 55, 8, 9, 55, 64, 75, 8, 60, 9, 33, 30, 53, 30, 52, 26, 8, 42, 9, 43, 13, 42, 76, 10, 9, 20, 43, 12, 50, 47, 23, 9, 26, 45, 31, 46, 9, 25, 60, 9]


[[78, 20, 9, 37, 54, 8, 66, 61, 40, 39], [59, 9], [9, 76, 65, 29, 62, 21, 30, 13, 43, 145], [43, 29, 58, 10, 10, 61, 52, 31, 34, 53], [14, 12, 69, 65, 9, 53], [9, 14, 9, 50], [50, 35, 12, 50, 15, 48, 31, 16, 26, 46], [65, 9, 18, 12, 41, 14], [47, 30, 12, 27, 69, 20, 42, 36, 44, 43], [8, 44, 9, 76], [55, 8, 9, 55, 64], [75, 8, 60, 9, 33, 30, 53, 30, 52, 26], [8, 42, 9, 43, 13, 42, 76, 10, 9, 20], [43, 12, 50, 47, 23, 9, 26, 45, 31, 46], [9, 25, 60, 9]]


In [10]:
print(new_vasp_calls)

[[40, 17, 7, 23, 24, 6, 46, 30, 66, 22], [19, 8], [16, 13, 35, 20, 40, 8, 67, 10, 14, 25], [14, 17, 27, 7, 8, 49, 36, 32, 8, 18], [10, 11, 37, 25, 10, 31], [6, 20, 8, 53], [39, 7, 27, 32, 16, 32, 14, 26, 16, 21], [25, 8, 22, 29, 22, 27], [36, 37, 17, 14, 19, 8, 37, 33, 20, 30], [7, 15, 8, 40], [27, 6, 6, 19, 18], [30, 7, 32, 6, 23, 25, 37, 37, 29, 20], [7, 28, 12, 30, 14, 19, 21, 26, 9, 28], [16, 10, 37, 13, 19, 10, 16, 21, 9, 41], [8, 31, 33, 10]]


In [11]:
print(similar_clus)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9), (1, 1), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 8), (2, 9), (3, 0), (3, 2), (3, 3), (3, 4), (3, 6), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 0), (5, 2), (5, 3), (6, 1), (6, 4), (6, 7), (6, 9), (7, 1), (7, 2), (7, 3), (8, 2), (8, 5), (8, 6), (8, 7), (9, 0), (9, 1), (9, 2), (9, 3), (10, 0), (10, 1), (10, 2), (10, 3), (11, 1), (11, 2), (11, 3), (11, 4), (11, 7), (11, 9), (12, 0), (12, 2), (12, 5), (12, 6), (12, 8), (13, 0), (13, 1), (13, 2), (13, 3), (13, 5), (13, 7), (14, 0), (14, 3)]


In [12]:
same_clus_almlp = []
same_clus_vasp = []
same_clus_diff = []
for i in range(len(similar_clus)):
    p = similar_clus[i][0]
    q = similar_clus[i][1]
    
    r = new_vasp_calls[p][q] #clus_XXX.log AL
    same_clus_almlp.append(r)
    s = vasp_calls_list[p][q] #results.log DFT
    same_clus_vasp.append(s)
    
    same_clus_diff.append(s-r)
    #print(r, s, s-r)
print(same_clus_almlp)
print(sum(same_clus_almlp))
print(same_clus_vasp)
print(sum(same_clus_vasp))
print(same_clus_diff)
print(sum(same_clus_diff))

[40, 17, 7, 23, 6, 30, 66, 22, 8, 16, 13, 20, 40, 8, 14, 25, 14, 27, 7, 8, 36, 18, 10, 11, 37, 25, 10, 6, 8, 53, 7, 16, 26, 21, 8, 22, 29, 17, 8, 37, 33, 7, 15, 8, 40, 27, 6, 6, 19, 7, 32, 6, 23, 37, 20, 7, 12, 19, 21, 9, 16, 10, 37, 13, 10, 21, 8, 10]
1295
[78, 20, 9, 37, 8, 61, 40, 39, 9, 9, 76, 29, 62, 21, 43, 145, 43, 58, 10, 10, 52, 53, 14, 12, 69, 65, 9, 9, 9, 50, 35, 15, 16, 46, 9, 18, 12, 12, 20, 42, 36, 8, 44, 9, 76, 55, 8, 9, 55, 8, 60, 9, 33, 30, 26, 8, 9, 42, 76, 9, 43, 12, 50, 47, 9, 45, 9, 9]
2208
[38, 3, 2, 14, 2, 31, -26, 17, 1, -7, 63, 9, 22, 13, 29, 120, 29, 31, 3, 2, 16, 35, 4, 1, 32, 40, -1, 3, 1, -3, 28, -1, -10, 25, 1, -4, -17, -5, 12, 5, 3, 1, 29, 1, 36, 28, 2, 3, 36, 1, 28, 3, 10, -7, 6, 1, -3, 23, 55, 0, 27, 2, 13, 34, -1, 24, 1, -1]
913
